# Inference
The topic of this notebook is to prototype the creation of an inference pipeline that will serve as a guideline for the actual system to be implemented.

## Idea
1. Go through a single scene of the FDST images.
2. Define zones of interest.
3. Warp the perspective of a single image to 4 different images (3 warped, 1 original).
4. Use the model to get the count of people in each segment of the image, as well as each perspective.
5. Average the number of the predicted count for each zone of interest.

## Motivation
Since in the original idea 4 cameras are going to be used to count the people in one hall the goal is to simulate that by wapring the perspective of each image. Before hand some zones of interest need to be defined (3 to begin with). Then for each defined zone average the infered count.

### Imports and Globals

In [1]:
import os
import sys
import cv2
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [4]:
from models.definitions.csr import CSRNet
from models.definitions.dsnet import DenseScaleNet

ImportError: cannot import name 'img_as_float2' from 'skimage' (/home/fury/anaconda3/envs/torch/lib/python3.7/site-packages/skimage/__init__.py)

In [2]:
BEST_MODEL_SAVE_PATH = 'models/best/'
CKP_MODEL_SAVE_PATH = 'models/checkpoints/'

#### Utilities
Some functions that provide various utilities

In [3]:
def load_model(model_filepath):
    '''
        Load the model from the filepath
    '''
    state = torch.load(checkpoint_filepath)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    epoch = state['epoch']
    return epoch, model, optimizer